In [207]:
import pandas as pd
import numpy as np
import requests
import json

# Create 3 Dataframes from the excellent  api from satnogs

Then combines Satelites with Transmitters...

In [213]:
data_modes=requests.get('https://db.satnogs.org/api/modes')
df_modes = pd.DataFrame.from_records(json.loads(data_modes.content))

data_satellites=requests.get('https://db.satnogs.org/api/satellites')
df_satellites = pd.DataFrame.from_records(json.loads(data_satellites.content))

data_transmitters=requests.get('https://db.satnogs.org/api/transmitters')
df_transmitters = pd.DataFrame.from_records(json.loads(data_transmitters.content))

df = df_transmitters.merge(df_satellites,on=['norad_cat_id'],how='left')

# Use the norad_cat_id as the index

We need to process per satellite, so the easiest way is to use the norad id, as an index key.

In [214]:
df.set_index(['norad_cat_id'],inplace=True)

# Remove the non alive ... records

In [216]:
# It is really this simple
df=df[df.alive == 1]


def f_check(f):
    """
    Remove the NaN values and check the data.
    :param f frequency
    returns: int value of 0 or the int value of the specified frequency in Hertz
    """
    try:
        if f>1:
            return int(f)
        else:
            return int(0)
    except:
        return int(0)
    
df['downlink_low_mhz'] =df.downlink_low.apply(lambda x: f_check(x))
df['downlink_high_mhz']=df.downlink_high.apply(lambda x: f_check(x))
df['uplink_low_mhz']   =df.uplink_low.apply(lambda x: f_check(x))
df['uplink_high_mhz']  =df.uplink_high.apply(lambda x: f_check(x))

# Sample trsp files

This is what we are trying to produce.
If a satellite had 6 possible modes - then all of these items would be in the <norad_cat_id>.mod file.


```
[Mode V Digitalker (Voices Messages and Telemetry)]
DOWN_LOW=145950000
MODE=FM
```

```
[Mode V Imaging (Robot 36 SSTV from onboard cameras)]
DOWN_LOW=145950000
MODE=SSTV in FM carrier
```

```
[Mode V Telemetry (1000 baud (400 baud backup))]
DOWN_LOW=145920000
MODE=BPSK 1000 bps
```

```
[Mode U/V (B) Linear Transponder]
UP_LOW=435742000
UP_HIGH=435758000
DOWN_LOW=145922000
DOWN_HIGH=145938000
INVERT=true
```

```
[Mode V TLM Beacon (CW-2, BPSK-1000)]
DOWN_LOW=145919000
MODE=CW
```

```
[Mode V TLM Beacon (CW-1, BPSK-400)]
DOWN_LOW=145939000
MODE=CW
```

In [162]:
pre_idx=-1
of=None
for index, record in df.iterrows():
    if index != pre_idx:
        if of:
            of.close()    
        of=open(f"{index}.trsp","wt")
        pre_idx=index
    if record['mode'] in ['CW','FM','FMN','USB']:
        #Mode more simple
        title=f"[Mode {record['mode']} {record['description']}]"
    else:
        #Mode Complex Modes
        title=f"[Mode {record['mode']} (Baud {record['baud']} {record['description']})]"
    of.write(f"{title}\n")
    
    # Invert Rule
    if record['invert']:
        of.write(f"INVERT=true\n")
        
    # Baud Rule
    if record['baud']>0.0:
        of.write(f"MODE={record['mode']} {record['baud']}\n")
    else:
        of.write(f"MODE={record['mode']}\n")
    
    #Downlink Rule
    if record['downlink_high_mhz'] == 0:
        #print(f"Uplink is {record['downlink_high_mhz']} ")
        of.write(f"DOWN_LOW={record['downlink_low_mhz']}\n")
    else:
        of.write(f"DOWN_HIGH={record['downlink_high_mhz']}\n")                            
        of.write(f"DOWN_LOW={record['downlink_low_mhz']}\n")  

    #Uplink Rule
    if record['uplink_high_mhz'] != 0:
        of.write(f"UP_LOW={record['uplink_high_mhz']}\n")
    if record['uplink_low_mhz'] != 0:
        of.write(f"UP_LOW={record['uplink_low_mhz']}\n") 
                 
    of.write("\n\n")

# Trsp done

We should now have the trsp files... they need to be moved to the gprdict dirctory. 

To do this please do a 

   cp *.trsp ~/.config/Gpredict/trsp/

And we should be good to go.


### Modules for GPredict 

This list of Satelites is too much for me to wade through ... So I need to break them down by Mode... 


This is what GPredict calls a module... 
```
[GLOBAL]
QTHFILE=Arenas.qth
SATELLITES=40905;41460
```

You should change the QTHFILE= to match with what your QTH Location name is set as. The default file suffix is *.qth*. You can see this in **~/.config/Gpredict/** the home locations are by default set there.


In [217]:
MyQth = "Areanas.qth"   
for mode in df['mode'].unique():
        norad_ids=[]
        for index,rec in df.iterrows():
              if rec['mode']==mode:
                  norad_ids.append(index)
        if norad_ids != [] and mode:
             file_name=mode[0].upper()+mode[1:].lower()
             with open(f"{file_name}.mod","wt") as mod_file:
                # We need this header
                # You will need a Qth of the same name ... or alter the code/gpredict
                mod_file.write("[GLOBAL]\nQTHFILE=Arenas.qth\nSATELLITES=")
                norad_ids = list(set(norad_ids))
                norad_ids.sort() #They may as well be Numeric in order
                sat_line=';'.join([str(i) for i in norad_ids])
                mod_file.write(f"{sat_line}\n")


### mov Files


At this point you have created some MOV files, these also need to be copied into your GPredict directory.

    cp *.mov ~/.config/GPredict/modules
    
You should now be able to open these in GPredict - grouped per transmission mode, and will full transponder data.